# Model

In [1]:
import sys
import os
!pip install -q pyomo
import pandas as pd
import pyomo.environ as pyo

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.0/305.0 kB 26.4 MB/s eta 0:00:00
Getting files...
Done
---------------------------------------------------
IDAES Extensions Build Versions
Solvers:  vno version file found
Library:  vno version file found


In [4]:
from google.colab import files
data = pd.read_csv('/may_2019_SAN_Airport.csv')

In [5]:
data.head(5)

,Unnamed: 0,Year,Quarter,Month,DayofMonth,FlightDate,Flight_Number_Reporting_Airline,OriginAirportID,DayOfWeek,Origin,...,Dest,DestCityName,DestState,DepTime,Flights,DepDelay,ArrDelay,ArrTime,DepTimeBlk,ArrTimeBlk
0,917840,2019,2,5,21,2019-05-21,1952,14771,2,SFO,...,SAN,"San Diego, CA",CA,843.0,1.0,-2.0,-5.0,1015.0,0800-0859,1000-1059
1,917841,2019,2,5,21,2019-05-21,1953,14679,2,SAN,...,SFO,"San Francisco, CA",CA,924.0,1.0,24.0,25.0,1105.0,0900-0959,1000-1059
2,917842,2019,2,5,21,2019-05-21,1954,14771,2,SFO,...,SAN,"San Diego, CA",CA,1147.0,1.0,7.0,-5.0,1310.0,1100-1159,1300-1359
3,917843,2019,2,5,21,2019-05-21,1957,14679,2,SAN,...,SFO,"San Francisco, CA",CA,1122.0,1.0,-8.0,2.0,1312.0,1100-1159,1300-1359
4,917844,2019,2,5,21,2019-05-21,1958,14771,2,SFO,...,SAN,"San Diego, CA",CA,1500.0,1.0,0.0,10.0,1645.0,1500-1559,1600-1659


In [6]:
list_of_flight = data[data['FlightDate'] == '2019-05-06']
list_of_arrivals = list_of_flight[list_of_flight['Dest'] == 'SAN']
list_of_departures = list_of_flight[list_of_flight['Origin'] == 'SAN']

In [22]:
len(list_of_departures) + len(list_of_arrivals)

540

In [8]:
departure_capacities = [0, 25, 21, 17, 17, 16, 16, 16, 16, 16, 15, 15, 15, 15, 15, 15, 15, 15, 15, 16]
arrival_capacities = [0 ,7, 11, 15, 15, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 16]

In [9]:
list_of_arrivals.head(10)
time_order = list_of_arrivals.sort_values('ArrTime', ascending=True)
time_order_departures = list_of_departures.sort_values('DepTime', ascending=True)

In [10]:
model = pyo.ConcreteModel()
I = (range(1,len(time_order)+1))
model.I = pyo.Set(initialize = I)

In [11]:
J = range(time_order['ArrTimeBlk'].nunique() + 2)
model.J = pyo.Set(initialize = J)

In [12]:
model.d_capacity = pyo.Param(model.J, initialize = departure_capacities)
model.a_capacity = pyo.Param(model.J, initialize = arrival_capacities)

In [13]:
model.d_capacity.pprint()

d_capacity : Size=20, Index=J, Domain=Any, Default=None, Mutable=False
    Key : Value
      0 :     0
      1 :    25
      2 :    21
      3 :    17
      4 :    17
      5 :    16
      6 :    16
      7 :    16
      8 :    16
      9 :    16
     10 :    15
     11 :    15
     12 :    15
     13 :    15
     14 :    15
     15 :    15
     16 :    15
     17 :    15
     18 :    15
     19 :    16


In [15]:
list_of_arrivals_per_time_block = []
counts = {}
for value in time_order['ArrTimeBlk']: # Incrementing for each variable, and starting a new count if the variable has not been seen yet
    if value in counts:
        counts[value] += 1
    else:
        counts[value] = 1
counts['excess'] = 0
counts['fixing_0'] = 0
list_of_departures_per_time_block = []
counts_departures = {}
for value in time_order_departures['DepTimeBlk']: # Incrementing for each variable, and starting a new count if the variable has not been seen yet
    if value in counts_departures:
        counts_departures[value] += 1
    else:
        counts_departures[value] = 1
counts_departures['excess'] = 0
counts_departures['fixing_0'] = 0
counts_departures['2300-2359'] = 0

In [16]:
counts_list = []
for i in counts:
  counts_list.append((i, counts[i]))

indexes = []
for i in counts:
  indexes.append(i)

counts_list_departures = []
for i in counts_departures:
  counts_list_departures.append((i, counts_departures[i]))

indexes_departures = []
for i in counts_departures:
  indexes_departures.append(i)


ordered_list_of_arrivals = sorted(counts_list, key = lambda x:x[0])

ordered_list_of_departures = sorted(counts_list_departures, key = lambda x:x[0])

indexes_sorted = sorted(indexes, key = lambda x:x)
indexes_sorted_departures = sorted(indexes_departures, key = lambda x:x)

In [17]:
print(ordered_list_of_departures)

[('0600-0659', 29), ('0700-0759', 20), ('0800-0859', 14), ('0900-0959', 21), ('1000-1059', 21), ('1100-1159', 16), ('1200-1259', 17), ('1300-1359', 15), ('1400-1459', 15), ('1500-1559', 14), ('1600-1659', 11), ('1700-1759', 16), ('1800-1859', 18), ('1900-1959', 6), ('2000-2059', 14), ('2100-2159', 11), ('2200-2259', 12), ('2300-2359', 0), ('excess', 0), ('fixing_0', 0)]


In [18]:
key_mapping = {'fixing_0':0,'0600-0659': 1, '0700-0759': 2, '0800-0859': 3, '0900-0959': 4, '1000-1059': 5, '1100-1159': 6, '1200-1259': 7, '1300-1359': 8, '1400-1459': 9, '1500-1559': 10, '1600-1659': 11, '1700-1759': 12, '1800-1859': 13, '1900-1959': 14, '2000-2059': 15, '2100-2159': 16, '2200-2259': 17, '2300-2359': 18,'excess':19 }

In [19]:
updated_dict = {key_mapping.get(old_key, old_key): value for old_key, value in ordered_list_of_arrivals}
updated_dict_departures = {key_mapping.get(old_key, old_key): value for old_key, value in ordered_list_of_departures}
updated_dict[0] = 0
print(updated_dict_departures)

{1: 29, 2: 20, 3: 14, 4: 21, 5: 21, 6: 16, 7: 17, 8: 15, 9: 15, 10: 14, 11: 11, 12: 16, 13: 18, 14: 6, 15: 14, 16: 11, 17: 12, 18: 0, 19: 0, 0: 0}


In [21]:
ordered_list_of_arrivals + ordered_list_of_departures

[('0600-0659', 3),
 ('0700-0759', 9),
 ('0800-0859', 18),
 ('0900-0959', 18),
 ('1000-1059', 19),
 ('1100-1159', 17),
 ('1200-1259', 17),
 ('1300-1359', 13),
 ('1400-1459', 16),
 ('1500-1559', 10),
 ('1600-1659', 15),
 ('1700-1759', 17),
 ('1800-1859', 11),
 ('1900-1959', 18),
 ('2000-2059', 21),
 ('2100-2159', 15),
 ('2200-2259', 14),
 ('2300-2359', 19),
 ('excess', 0),
 ('fixing_0', 0),
 ('0600-0659', 29),
 ('0700-0759', 20),
 ('0800-0859', 14),
 ('0900-0959', 21),
 ('1000-1059', 21),
 ('1100-1159', 16),
 ('1200-1259', 17),
 ('1300-1359', 15),
 ('1400-1459', 15),
 ('1500-1559', 14),
 ('1600-1659', 11),
 ('1700-1759', 16),
 ('1800-1859', 18),
 ('1900-1959', 6),
 ('2000-2059', 14),
 ('2100-2159', 11),
 ('2200-2259', 12),
 ('2300-2359', 0),
 ('excess', 0),
 ('fixing_0', 0)]

In [ ]:
model.A = pyo.Param(model.J, initialize = updated_dict)
model.D = pyo.Param(model.J, initialize = updated_dict_departures)

In [ ]:
model.D.pprint()

D : Size=20, Index=J, Domain=Any, Default=None, Mutable=False
    Key : Value
      0 :     0
      1 :    29
      2 :    20
      3 :    14
      4 :    21
      5 :    21
      6 :    16
      7 :    17
      8 :    15
      9 :    15
     10 :    14
     11 :    11
     12 :    16
     13 :    18
     14 :     6
     15 :    14
     16 :    11
     17 :    12
     18 :     0
     19 :     0


In [ ]:
model.s = pyo.Var(model.J, domain = pyo.NonNegativeIntegers)
model.k = pyo.Var(model.J, domain= pyo.NonNegativeIntegers)

In [ ]:
def obj_rule(model):
  return sum(1000 * model.s[i] for i in model.J)

model.obj = pyo.Objective(rule = obj_rule, sense = pyo.minimize)
model.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : 1000*s[0] + 1000*s[1] + 1000*s[2] + 1000*s[3] + 1000*s[4] + 1000*s[5] + 1000*s[6] + 1000*s[7] + 1000*s[8] + 1000*s[9] + 1000*s[10] + 1000*s[11] + 1000*s[12] + 1000*s[13] + 1000*s[14] + 1000*s[15] + 1000*s[16] + 1000*s[17] + 1000*s[18] + 1000*s[19]


In [ ]:
indexes = range(1,time_order['ArrTimeBlk'].nunique() + 2)
model.Indexes = pyo.Set(initialize = indexes)
def Constraint_rule1(model, i):
  return(model.A[i] + model.s[i-1]+ model.D[i] - model.s[i] <= model.a_capacity[i]+ model.d_capacity[i])
model.Constraint1 = pyo.Constraint(model.Indexes, rule = Constraint_rule1)

In [ ]:
solver = pyo.SolverFactory("cbc")
results = solver.solve(model)
results.write()
pyo.value(model.obj)
print(model.s.pprint())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 92000.0
  Upper bound: 92000.0
  Number of objectives: 1
  Number of constraints: 19
  Number of variables: 20
  Number of binary variables: 0
  Number of integer variables: 20
  Number of nonzeros: 20
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bou